In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim

## 0. Dataset and DataLoader

In [ ]:
class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)
        
        # 采样每5行取一行
        self.data = self.data.iloc[::5, :].reset_index(drop=True)
        
        # 计算原始数据的标准差
        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()
        
        # 初始化归一化器
        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()
            
            # 对风功率数据进行归一化
            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            
            # 对天气数据进行归一化
            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)
        future_wind_power = self.data.iloc[idx + 312, 2]
        return torch.tensor(wind_power_history, dtype=torch.float32), \
               torch.tensor(future_weather, dtype=torch.float32), \
               torch.tensor(future_wind_power, dtype=torch.float32)
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }

In [ ]:
dataset = WindPowerDataset('CAISO_zone_1_.csv', save_scalers=True)
weather_stds = dataset.get_original_stds()
total_size = len(dataset)
train_size = int(0.8 * total_size)  
test_size = total_size - train_size  
train_idx = list(range(train_size))
test_idx = list(range(train_size, total_size))

train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   

wind_power_scaler=dataset.wind_power_scaler

## 1. UP

In [ ]:
def optimize_universal_perturbation(model, data_loader, epsilon=0.01, device='cpu', num_epochs=100, lr=0.01, lambda_reg=0.0001):
    model.train()  # 将模型设置为训练模式

    # 冻结模型参数
    for param in model.parameters():
        param.requires_grad = False

    # 初始化通用扰动向量
    perturbation_shape = (24, 8)  # 与输入数据的 weather_future 形状一致
    universal_perturbation = torch.tensor(torch.rand(perturbation_shape, device=device) * 2 * epsilon - epsilon, requires_grad=True)

    # 定义优化器
    optimizer = optim.Adam([universal_perturbation], lr=lr)
    criterion = nn.MSELoss()

    best_perturbation = universal_perturbation.clone().detach()
    best_loss = float('inf')

    for epoch in range(num_epochs):
        total_loss = 0
        for wind_history, weather_future, future_wind_power in data_loader:
            wind_history, weather_future, future_wind_power = wind_history.to(device), weather_future.to(device), future_wind_power.to(device)
            
            # 生成对抗样本
            weather_future_adversarial = weather_future + universal_perturbation.unsqueeze(0).expand_as(weather_future)
            weather_future_adversarial = torch.clamp(weather_future_adversarial, 0, 1)  # 保持数据在归一化范围内
            
            # 计算对抗样本的预测
            output_adversarial = model(wind_history, weather_future_adversarial)
            
            # 计算损失，包括L1正则化项
            #loss = -criterion(output_adversarial.squeeze(), future_wind_power)  # 希望误差最大化，所以取负值
            loss = -torch.sum(output_adversarial-future_wind_power)
            #l1_reg = lambda_reg * torch.norm(universal_perturbation, p=1)  # L1正则化项
            total_loss = loss 
            
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            
            # 裁剪扰动向量，使其在 [-epsilon, epsilon] 之间
            with torch.no_grad():
                universal_perturbation.clamp_(-epsilon, epsilon)
            
            total_loss += total_loss.item()
        
        # 更新最优扰动向量
        if total_loss < best_loss:
            best_loss = total_loss
            #将universal_perturbation进行深拷贝得到best_perturbation
            best_perturbation = universal_perturbation.clone().detach()
        average_loss = total_loss / len(data_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}')
        best_average_loss = best_loss / len(data_loader)
    print(f'Best average loss: {best_average_loss:.4f}')
    print('---------------------')
    
    # 恢复模型参数的梯度设置
    for param in model.parameters():
        param.requires_grad = True

    return best_perturbation

### UP-LSTM

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model =WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model_path = 'wind_caiso_lstm_sigmoid.pth' 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model(model_path, device)

In [ ]:
uap_list010=[]
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.1, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list010.append(universal_perturbation)

In [ ]:
uap_list005=[]
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.05, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list005.append(universal_perturbation)

In [ ]:
uap_list003=[]
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.03, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list003.append(universal_perturbation)

### UP-Transformer

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  # 应用Sigmoid激活函数
        return output

In [ ]:
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
# 调用加载模型的函数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = 'wind_caiso_transformer_sigmoid.pth'
model = load_model(model_path, device)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.1, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list010.append(universal_perturbation)


In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.05, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list005.append(universal_perturbation)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.03, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list003.append(universal_perturbation)

### UP-GRU

In [ ]:
class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=50, num_layers=2, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=50, num_layers=2, batch_first=True)  
        self.fc = nn.Linear(100, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  
model_path = 'wind_gru_caiso_sigmoid.pth'
model = load_model(model_path, device)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.1, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list010.append(universal_perturbation)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.05, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list005.append(universal_perturbation)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.03, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list003.append(universal_perturbation)

### UP-TCN

In [ ]:
model_path = 'wind_tcn_caiso_sigmoid.pth'
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1) * dilation_size, dilation=dilation_size),
                       nn.ReLU(),
                       nn.Dropout(dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class WindPowerPredictorTCN(nn.Module):
    def __init__(self):
        super(WindPowerPredictorTCN, self).__init__()
        self.tcn_wind = TemporalConvNet(num_inputs=1, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.tcn_weather = TemporalConvNet(num_inputs=8, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.fc = nn.Linear(50 * 2, 1)  # Combined output size of TCNs
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(1)  # (batch_size, 1, seq_len)
        tcn_output_wind = self.tcn_wind(wind_history).transpose(1, 2)[:, -1, :]
        
        weather_future = weather_future.transpose(1, 2)  # (batch_size, 8, seq_len)
        tcn_output_weather = self.tcn_weather(weather_future).transpose(1, 2)[:, -1, :]
        
        combined = torch.cat((tcn_output_wind, tcn_output_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
def load_model(model_path, device='cpu'):
    model = WindPowerPredictorTCN().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model
model = load_model(model_path, device)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.1, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list010.append(universal_perturbation)

with open('uap_list010_caiso.pkl', 'wb') as f:
    pickle.dump(uap_list010, f)


In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.05, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list005.append(universal_perturbation)

with open('uap_list005_caiso.pkl', 'wb') as f:
    pickle.dump(uap_list005, f)

In [ ]:
for i in range(20):
    universal_perturbation = optimize_universal_perturbation(model, train_loader, epsilon=0.03, device=device, num_epochs=20, lr=0.00001, lambda_reg=0)
    uap_list003.append(universal_perturbation)

with open('uap_list003_caiso.pkl', 'wb') as f:
    pickle.dump(uap_list003, f)

## RUP

In [ ]:
def adversarial_attack_inde(uap_loaded, model, data_loader,  device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)

        original_output = model(wind_history, weather_future)

        adversarial_weather = weather_future + uap_loaded
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  

        adversarial_output = model(wind_history, adversarial_weather.detach())  

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = 'wind_caiso_lstm_sigmoid_version2.pth'   
test_model = load_model(model_path, device)

In [ ]:
mae_list010 = []
for uap_loaded in uap_list010:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, test_model, test_loader, device=device)
    adversarial_preds_uni010 = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv010=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni010).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv010)
    r2= r2_score(all_targets_inv, all_preds_inv010)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv010, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)
    mae_list010.append(mae)
    print('---------------------')

In [ ]:
mae_list005 = []
for uap_loaded in uap_list005:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, test_model, test_loader, device=device)
    adversarial_preds_uni005 = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv005=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni005).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv005)
    r2= r2_score(all_targets_inv, all_preds_inv005)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv005, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)
    mae_list005.append(mae)
    print('---------------------')
    

In [ ]:
mae_list003 = []
for uap_loaded in uap_list003:
    original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_inde(uap_loaded, test_model, test_loader, device=device)
    adversarial_preds_uni003 = np.concatenate(adversarial_preds).flatten()
    targets= np.concatenate(targets).flatten()
    all_targets_inv=wind_power_scaler.inverse_transform(np.array(targets).reshape(-1, 1)).squeeze()
    all_preds_inv003=wind_power_scaler.inverse_transform(np.array(adversarial_preds_uni003).reshape(-1, 1)).squeeze()
    mae = mean_absolute_error(all_targets_inv, all_preds_inv003)
    r2= r2_score(all_targets_inv, all_preds_inv003)
    rmse=mean_squared_error(all_targets_inv, all_preds_inv003, squared=False)
    print('r2=',r2)
    print('rmse=',rmse)
    print('mae=',mae)
    mae_list003.append(mae)
    print('---------------------')

In [ ]:
with open('mae_list003_caiso.pkl', 'wb') as f:
    pickle.dump(mae_list003, f)

with open('mae_list005_caiso.pkl', 'wb') as f:
    pickle.dump(mae_list005, f)

with open('mae_list010_caiso.pkl', 'wb') as f:
    pickle.dump(mae_list010, f)

In [ ]:
from scipy.spatial.distance import cdist

def calculate_local_density(uaps, sigma=1.0):
    uaps_flat = uaps.reshape(uaps.shape[0], -1)  
    distances = cdist(uaps_flat, uaps_flat, 'euclidean')
    exp_distances = np.exp(-distances**2 / (2 * sigma**2))
    densities = np.sum(exp_distances, axis=1) - 1  
    return densities

# WD_k,q
def calculate_weighted_density(densities, deviations):
    # ρ_k,q * Dev_k,q
    weighted_products = densities * deviations   
    total_weighted_product = np.sum(weighted_products)
    weighted_densities = weighted_products / total_weighted_product
    
    return weighted_densities

def combine_ups(uaps, weighted_densities):
    rup = np.sum([density * up for density, up in zip(weighted_densities, uaps)], axis=0)
    return rup

configurations = [
    ('010', uap_list010, mae_list010),
    ('005', uap_list005, mae_list005),
    ('003', uap_list003, mae_list003),
]

for suffix, uap_list, mae_list in configurations:
    uaps = np.array([uap.cpu().numpy() for uap in uap_list])
    deviations = np.array(mae_list)
    local_densities = calculate_local_density(uaps, sigma=0.02)  #using different sigma could get different results
    weighted_densities = calculate_weighted_density(local_densities, deviations)
    rup_ensemble = combine_ups(uaps, weighted_densities)
    
    filename = f'rup_ensemble_{suffix}_caiso.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(rup_ensemble, f)

## RUPW

In [ ]:
def simple_average_ups(uap_list):
    uaps_numpy = [uap.cpu().numpy() for uap in uap_list]
    uaps_stack = np.stack(uaps_numpy, axis=0)  
    return np.mean(uaps_stack, axis=0)

rup_simple_avg010 = simple_average_ups(uap_list010)
rup_simple_avg005 = simple_average_ups(uap_list005)
rup_simple_avg003 = simple_average_ups(uap_list003)
with open('RUPW010_caiso.pkl', 'wb') as f:
    pickle.dump(rup_simple_avg010, f)
with open('RUPW005_caiso.pkl', 'wb') as f:
    pickle.dump(rup_simple_avg005, f)
with open('RUPW003_caiso.pkl', 'wb') as f:
    pickle.dump(rup_simple_avg003, f)